In [2]:
import sys
sys.path.append("../")

In [3]:
import QuantLib as ql
import numpy as np
import pandas as pd
from datetime import datetime, timedelta 
from typing import Dict
from pandas.tseries.offsets import CustomBusinessDay 
from pandas.tseries.holiday import USFederalHolidayCalendar

from CurvyCUSIPs.CurveInterpolator import GeneralCurveInterpolator
from CurvyCUSIPs.CurveDataFetcher import CurveDataFetcher
from CurvyCUSIPs.USTs import USTs
from CurvyCUSIPs.S490Swaps import S490Swaps
from CurvyCUSIPs.S490Swaptions import S490Swaptions 
from CurvyCUSIPs.utils.dtcc_swaps_utils import datetime_to_ql_date, ql_date_to_datetime
from CurvyCUSIPs.utils.regression_utils import run_multiple_linear_regression_df, run_basic_linear_regression_df

from CurvyCUSIPs.HedgeHog.beta import beta_estimates
from CurvyCUSIPs.HedgeHog.usts import dv01_neutral_curve_hedge_ratio, dv01_neutral_butterfly_hedge_ratio
import CurvyCUSIPs.HedgeHog.swaps as hh_swaps
import CurvyCUSIPs.HedgeHog.swaptions as hh_swaptions

import os
from dotenv import dotenv_values
env_path = os.path.join(os.getcwd(), "../.env")
config = dotenv_values(env_path)

import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
plt.style.use('ggplot')
params = {'legend.fontsize': 'large',
        'figure.figsize': (15, 9),
        'axes.labelsize': 'large',
        'axes.titlesize':'large',
        'xtick.labelsize':'large',
        'ytick.labelsize':'large'}
pylab.rcParams.update(params)

import nest_asyncio
nest_asyncio.apply()

%load_ext autoreload
%autoreload 2

In [4]:
curve_data_fetcher = CurveDataFetcher()

In [5]:
s490_swaps = S490Swaps(s490_curve_db_path=r"..\db\nyclose_sofr_ois", curve_data_fetcher=curve_data_fetcher)

start_date = datetime(2021, 1, 7)
end_date = datetime(2025, 1, 7)
bdates = pd.date_range(start=start_date, end=end_date, freq=CustomBusinessDay(calendar=USFederalHolidayCalendar()))

In [6]:
fwd_tenors = ["1M", "3M", "6M", "9M", "12M", "18M", "2Y", "3Y", "4Y", "5Y", "6Y", "7Y", "8Y", "9Y", "10Y"]
fwd_grid_dict, ql_curves = s490_swaps.s490_nyclose_fwd_curve_matrices(
    start_date=start_date,
    end_date=end_date,
    ql_piecewise_method="logLinearDiscount",
    fwd_tenors=fwd_tenors,
)

Building Implied Fwd Curves...: 100%|██████████| 1000/1000 [00:15<00:00, 64.15it/s]


# Rec Fixed 10MM in 3M Fwd 10Y on 07/22/2024:

<img src="../dump/j7DRPYFd.png" alt="drawing" height="750"/>


## Grab Market Data and Discount Curve on July 22, 2024:

In [116]:
date = datetime(2024, 7, 22)
fwd_grid_df = fwd_grid_dict[date]
ql_curve = ql_curves[date]

In [106]:
tenor = "3M Fwd 10Y" 

split = tenor.split(" Fwd ")
if len(split) == 1:
    fwd_tenor = "Spot"
    swap_tenor = split[0]
else:
    fwd_tenor, swap_tenor = split
    fwd_tenor = f"{fwd_tenor} Fwd" 

book = [
    hh_swaps.SwapLeg(
        trade_date=date,
        original_tenor=tenor,
        original_fixed_rate=fwd_grid_df[fwd_grid_df["Tenor"] == swap_tenor][fwd_tenor].iloc[-1] / 100,
        weighting=1,
        key=tenor,
        notional=10_000_000,
        type="receiver",
    )
]

In [107]:
swap_book_metrics = hh_swaps.book_metrics(swap_portfolio=book, ql_curve=ql_curve, ql_yts=s490_swaps._ql_yts, ql_sofr=s490_swaps._ql_sofr)
swap_book_metrics

{'total_carry_and_roll': None,
 'bps_running': {'3M Fwd 10Y': {'roll': {'30D': 0.634625314846482,
    '60D': 0.7257397027591456,
    '90D': 0.799814266251378,
    '180D': 0.9943627782366127,
    '360D': 0.9238772081486546},
   'carry': {'30D': -2.9103069650918827,
    '60D': -5.087261709264476,
    '90D': -6.4682227394241565,
    '180D': -9.724101969037571,
    '360D': -10.375495786819691}}},
 'book': {'3M Fwd 10Y': <QuantLib.QuantLib.OvernightIndexedSwap; proxy of <Swig Object of type 'ext::shared_ptr< OvernightIndexedSwap > *' at 0x00000223D52B0C00> >},
 'book_bps': 8171.56263893661}

In [108]:
ql_ois: ql.OvernightIndexedSwap = swap_book_metrics["book"]["3M Fwd 10Y"]

In [115]:
ql.Settings.instance().evaluationDate = datetime_to_ql_date(datetime(2024, 7, 22)) 
ql_ois.NPV(), ql_ois.legNPV(1), ql_ois.fixedLegNPV(), ql_ois.floatingLegNPV(), ql_ois.overnightLegNPV(), ql_ois.fixedLegBPS()

(-5667.32176798908,
 -3085031.7462708508,
 3079364.4245028617,
 -3085031.7462708508,
 -3085031.7462708508,
 8171.56263893661)

In [114]:
data = []
for cf in ql_ois.overnightLeg():
    coupon: ql.FloatingRateCoupon = ql.as_floating_rate_coupon(cf)
    data.append(
        (
            coupon.date(),
            coupon.nominal(),
            coupon.rate(),
            coupon.accrualPeriod(),
            coupon.amount(),
            coupon.price(ql.YieldTermStructureHandle(ql_curve)),
        )
    )

pd.DataFrame(
    data, columns=["date", "nominal", "rate", "tenor", "amount", "price"]
).style.format({"amount": "{:.2f}", "rate": "{:.2%}", "nominal": "{:.0f}"})

,date,nominal,rate,tenor,amount,price
0,"October 28th, 2025",10000000,4.43%,1.013889,448943.33,423651.463608
1,"October 28th, 2026",10000000,3.76%,1.019444,383421.15,348534.633663
2,"October 27th, 2027",10000000,3.55%,1.011111,358924.00,314962.190472
3,"October 26th, 2028",10000000,3.54%,1.013889,358604.04,303786.860966
4,"October 26th, 2029",10000000,3.59%,1.013889,363736.18,297318.484365
5,"October 28th, 2030",10000000,3.66%,1.013889,371283.82,292563.960023
6,"October 28th, 2031",10000000,3.72%,1.013889,376803.53,286128.177612
7,"October 27th, 2032",10000000,3.81%,1.019444,388006.03,283688.704476
8,"October 26th, 2033",10000000,3.81%,1.011111,384774.07,270902.063785
9,"October 26th, 2034",10000000,3.83%,1.013889,388807.46,263495.207301


## Why is NPV off?

- notice that my 3m fwd 10y rate was 3.7683 and BBG was showing 3.736141
- Swap NPV calculation seems to be ill-conditioned but 3bps is signficant and notional was big
- Still good enough for something that is free lol